In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=True,
                                        download=True, transform=transform)
#trainset.data = trainset.data[:]
#trainset.targets = trainset.targets[:]
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from rac.utils.models import cifar10net, resnet

In [4]:
import torch.nn as nn
import torch.nn.functional as F

In [20]:
model1 = resnet.ResNet18()
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    outputs, e = model1(inputs, last=True)
    print(outputs.shape, e.shape)
    break

torch.Size([128, 10]) torch.Size([128, 512])


In [11]:
model1 = cifar10net.CifarNet()
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    outputs, e = model1(inputs, last=True)
    print(outputs.shape, e.shape)
    break

torch.Size([256, 10]) torch.Size([256, 256])


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
device

device(type='cuda', index=0)

In [9]:
import torch.optim as optim
model1 = cifar10net.CifarNet().to(device)
#model1 = resnet.ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(100):  # loop over the dataset multiple times
    print(epoch)

    running_loss = 0.0
    step = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device),

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        step += 1
    print("loss: ", running_loss/step)
    step = 0
    running_loss = 0.0
print('Finished Training')

0
loss:  2.296664716642531
1
loss:  2.2365966172474425
2
loss:  2.03174294199785
3
loss:  1.9081820999569905
4
loss:  1.7904420981321798
5
loss:  1.715658109816139
6
loss:  1.6478865817379769
7
loss:  1.5760574374357452
8
loss:  1.5115844840588777
9
loss:  1.455801753436818
10
loss:  1.4060164504039012
11
loss:  1.3685591449518033
12
loss:  1.32496985724515
13
loss:  1.2915046577868254
14
loss:  1.2607482848569864
15
loss:  1.2300487753680296
16
loss:  1.1966816587826175
17
loss:  1.1678809767488934
18
loss:  1.1500399871860318
19
loss:  1.1133978112274423
20
loss:  1.0983266359399957
21
loss:  1.0671409903584843
22
loss:  1.044033279046988
23
loss:  1.019218028353913
24
loss:  1.0086806829628128
25
loss:  0.9854861558855647
26
loss:  0.9690406677668052
27
loss:  0.9543325539745028
28
loss:  0.9363260459716972
29
loss:  0.914919696195656
30
loss:  0.907658660045975
31
loss:  0.8953567657934125
32
loss:  0.8730667513959548
33
loss:  0.8690412764049247
34
loss:  0.8475872218761298
35
los

In [11]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = model1(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 79 %
